In [2]:
'''import relevant libraries'''
import os
import numpy as np
import string
from keras import models, layers, regularizers, preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers import Embedding, Flatten, Dense
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt 
from matplotlib import style
import pandas as pd 
import seaborn as sns
import re
import matplotlib.pyplot as partial_train_data
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression,Perceptron,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score,cross_val_predict,train_test_split, KFold
from sklearn.metrics import confusion_matrix,precision_score,recall_score,roc_auc_score,precision_recall_curve,roc_curve, accuracy_score
from sklearn.preprocessing import OneHotEncoder,StandardScaler, MinMaxScaler


In [3]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(128,kernel_regularizer=regularizers.l2(0.003), activation='relu', input_shape=(117,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16,kernel_regularizer=regularizers.l2(0.003),activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(16,kernel_regularizer=regularizers.l2(0.002),activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    
    return model

In [4]:

def kfoldvalidation():
    train_targets = pd.read_csv('train_targets.csv', delimiter=',', engine='python')
    train_data = pd.read_csv('train_data.csv', delimiter=',', engine='python')
    one_hot = pd.get_dummies(train_targets, prefix= '_')
    train_targets = pd.concat([train_targets,one_hot],axis=1)
    train_targets = train_targets.drop(['Degree of the injury'], axis=1)
    train_data = train_data.drop(['Accident Description'], axis=1) 
    print(train_targets.head(10))
    
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_targets, test_size=0.1, random_state=70)

    model = build_model()

    model.fit(X_train, y_train, epochs=100, batch_size=20, verbose=1)

    results = model.evaluate(X_test, y_test, verbose=0)

    print(results)
    
    
    '''
    neural_network = KerasClassifier(build_fn = build_model, epochs=80, batch_size=50, verbose=1) 
    results = cross_val_score(estimator=neural_network, X=train_data, y=train_targets, cv=6)

    print(results)
    print(results.mean())
    print(results.std())'''

    return results

In [5]:

def embeddingwords():
    maxlen = 1000
    training_samples = 200
    validation_samples = 10000
    max_words = 10000

    train_data = pd.read_csv('Data/accident_data.csv', delimiter=',', engine='python')
    train_data = train_data['Accident Description']

    train_data = train_data.str.replace('([\d]+)\.','')
    train_data = train_data.str.lstrip()
    
    print(train_data[0:5])
    train_targets = pd.read_csv('train_targets3.csv', delimiter= ',', engine='python')
    one_hot = pd.get_dummies(train_targets, prefix='Degree')
    train_targets = pd.concat([train_targets, one_hot], axis=1)
    train_targets = train_targets.drop(['Degree of the injury'], axis=1)

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_data)
    sequences = tokenizer.texts_to_sequences(train_data)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    train_data = pad_sequences(sequences, maxlen=maxlen)
    
    print('Shape of data tensor:', train_data.shape)
    #train_data = preprocessing.sequence.pad_sequences(train_data, maxlen=max_len, padding='post')
    model = Sequential()
    model.add(Embedding(10000, 8, input_length = maxlen))

    model.add(Flatten())
    model.add(Dense(36, activation='relu'))
    model.add(layers.Dropout(0.6))
    model.add(Dense(16, activation='relu'))
    model.add(layers.Dropout(0.6))
    model.add(layers.Dense(4,activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    history = model.fit(train_data,train_targets, epochs=20, batch_size=60, validation_split=0.1)

In [6]:
'''Run this to obtain model summarry and accuracy'''
'''Two models can be used. I suggest kfoldvalidation(), accuracies is better '''
kfoldvalidation()
#embeddingwords()


W0228 18:36:46.617911 13260 module_wrapper.py:139] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



   __Death  __Minor  __Moderate  __Severe
0        0        1           0         0
1        0        1           0         0
2        0        0           0         1
3        0        1           0         0
4        0        0           1         0
5        0        1           0         0
6        0        1           0         0
7        0        0           1         0
8        0        0           1         0
9        0        1           0         0


W0228 18:36:46.785988 13260 module_wrapper.py:139] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0228 18:36:47.042120 13260 module_wrapper.py:139] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0228 18:36:47.128936 13260 module_wrapper.py:139] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0228 18:36:47.138931 13260 deprecation.py:506] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `ra

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               15104     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 68        
Total para

W0228 18:36:47.690698 13260 deprecation.py:323] From c:\python37\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0228 18:36:48.091939 13260 module_wrapper.py:139] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0228 18:36:48.105931 13260 module_wrapper.py:139] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0228 18:36:48.278842 13260 module_wrapper.py:139] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0228 18:36:48.288828 13260 module_wrapper.py:139] From c:\p

Epoch 1/100


W0228 18:36:48.956602 13260 module_wrapper.py:139] From c:\python37\lib\site-packages\keras\backend\tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



1957/1957 [==============================] - 2s 986us/step - loss: 1.5453 - acc: 0.5013
Epoch 2/100
1957/1957 [==============================] - 0s 189us/step - loss: 1.3405 - acc: 0.5365
Epoch 3/100
1957/1957 [==============================] - 0s 157us/step - loss: 1.2407 - acc: 0.5575
Epoch 4/100
1957/1957 [==============================] - 0s 152us/step - loss: 1.1769 - acc: 0.5738
Epoch 5/100
1957/1957 [==============================] - 0s 166us/step - loss: 1.1228 - acc: 0.5759
Epoch 6/100
1957/1957 [==============================] - 0s 176us/step - loss: 1.0923 - acc: 0.5825
Epoch 7/100
1957/1957 [==============================] - 0s 154us/step - loss: 1.0521 - acc: 0.5856
Epoch 8/100
1957/1957 [==============================] - 0s 179us/step - loss: 1.0446 - acc: 0.6019
Epoch 9/100
1957/1957 [==============================] - 0s 149us/step - loss: 1.0255 - acc: 0.5994
Epoch 10/100
1957/1957 [==============================] - 0s 188us/step - loss: 0.9983 - acc: 0.6122
Epoch 11/10

1957/1957 [==============================] - 0s 147us/step - loss: 0.9215 - acc: 0.6413
Epoch 81/100
1957/1957 [==============================] - 0s 125us/step - loss: 0.9299 - acc: 0.6500
Epoch 82/100
1957/1957 [==============================] - 0s 168us/step - loss: 0.9099 - acc: 0.6387
Epoch 83/100
1957/1957 [==============================] - 0s 181us/step - loss: 0.9156 - acc: 0.6474
Epoch 84/100
1957/1957 [==============================] - 0s 189us/step - loss: 0.9217 - acc: 0.6484
Epoch 85/100
1957/1957 [==============================] - 0s 166us/step - loss: 0.9246 - acc: 0.6433
Epoch 86/100
1957/1957 [==============================] - 0s 174us/step - loss: 0.9217 - acc: 0.6428
Epoch 87/100
1957/1957 [==============================] - 0s 184us/step - loss: 0.9241 - acc: 0.6464
Epoch 88/100
1957/1957 [==============================] - 0s 146us/step - loss: 0.9245 - acc: 0.6428
Epoch 89/100
1957/1957 [==============================] - 0s 127us/step - loss: 0.9294 - acc: 0.6336
Epo

[0.9039710650750257, 0.701834864025816]